In [1]:
import pandas as pd
import numpy as np
import pickle

# 제가 임의로 만든 dummy data를 불러옵니다. (유저-카테고리-조회수)
dataframes = pd.read_pickle('recom_data/user_category_dummy3.pkl')
# print(dataframes)

df_counts = dataframes['view_counts']
df_users = dataframes['users']

# key: 카테고리 no / values: 카테고리명으로 구성된 dictionary입니다.
with open('recom_data/category_dict.pkl', 'rb') as handle:
    category_dict = pickle.load(handle)

In [2]:
# dummy data에 저희 DB에 저장된 유저-카테고리-조회수 데이터를 추가합니다.
db_view_data = [[2, 11, 1], [2, 12, 2], [2, 1, 1]]
df_db_view_data = pd.DataFrame(db_view_data, columns=['user', 'category', 'count'])

# dummy data에 저희 DB에 저장된 유저 정보를 추가합니다.
db_user_data = [[2, 'M', 27]]
df_db_user_data = pd.DataFrame(db_user_data, columns=['user', 'gender', 'age'])

# 지금부터 모든 계산은 dummy data + DB 데이터를 기준으로 진행됩니다. (new_df)
new_df_counts = df_counts.append(df_db_view_data)
new_df_users = df_users.append(df_db_user_data)

# print(new_df_counts)
# print(new_df_users)

In [3]:
# index: user / columns: category / values: count인 ... 무언가를 만듭니다. (출력해보세요)
view_matrix = new_df_counts.pivot(index='user', columns='category', values='count')
# print(view_matrix)

In [4]:
def transform_gender(gender):
    """
        gender를 M 또는 F로 변경합니다. (남자 / 여자 => M / F)
        이건 사실 제가 데이터를 잘못 만들어서... ... ㅎㅎ 나중에 수정할게용
    """
    return 'M' if (gender == '남자') else 'F'

def transform_age(age):
    """
        유저의 실제 나이를 연령대로 변환합니다. (17 => 10 / 21 => 20)
    """
    return (age // 10) * 10

In [5]:
def recom_simple1(user, n_items=20):
    # 유저 성별, 나이를 적절한 형태로 변환합니다.
    user_gender = transform_gender(user['gender'])
    user_age = transform_age(user['age'])

    # 유저와 같은 성별, 나이의 데이터를 필터링합니다.
    mask_gender = new_df_users['gender'] == user_gender
    mask_age = new_df_users['age'] == user_age
    
    user_group = new_df_users[mask_gender & mask_age]
    
    mask_group = new_df_counts['user'].isin(user_group['user'])
    # view_df는 유저와 같은 성별, 나이의 유저들의 카테고리별 조회수입니다.
    view_df = new_df_counts[mask_group]
    # print(view_df)
    
    # 데이터를 category를 기준으로 그룹화하고, 총합을 계산합니다.
    # 총합을 계산한 뒤 user의 숫자로 나누어서 유저별 평균 조회수를 계산합니다.
    # 만약 mean을 쓰면... .. 유저별 평균 조회수가 아니라 존재하는 데이터의 평균이 나올겁니다.
    # Ex) A가 a를 1번, b를 2번 조회 / B가 a를 1번, d를 3번 조회할 경우
    # sum으로 하고 나눈 경우 : a-1 / b-1 / d-1.5
    # mean으로 한 경우 : a-1 / b-2 / d-3
    grouped_view_df = (view_df.groupby(['category'])['count']
                       .agg(['sum'])
                       .sort_values(by=['sum'], ascending=False)
                       .apply(lambda x: x / len(view_df['user'].unique()))
                       .reset_index())
    
    return grouped_view_df

# user_id는 1인 27세 남자 데이터입니다.
# 아무 조회 기록이 없는 유저입니다!!
tester1 = {'user_id': 1, 'gender': '남자', 'age': 27}

# recom_simple1을 그냥 출력하면 category명이 아닌, category_no가 나옵니다.
# category 명이 출력되도록 해줍시다.
result = recom_simple1(tester1)
result['category'] = result['category'].apply(lambda x: category_dict[x])
print(result)

       category     sum
0        스마트 밴드  1.4500
1        스마트 워치  1.2500
2       남성 캐주얼화  1.1500
3        남성 운동화  1.1500
4        남성용 모자  1.1000
..          ...     ...
144  기타 스케이트 용품  0.2125
145       남성 향수  0.2125
146         전동휠  0.2000
147          목공  0.2000
148  기타 헤어 스타일링  0.1375

[149 rows x 2 columns]


In [6]:
def recom_simple2(user, n_items=20):
    # 기본적으로 그룹화 추천 결과를 가져옵니다.
    grouped_view_df = recom_simple1(user, n_items=n_items).set_index('category')
    
    # 현재 유저의 데이터를 DataFrame으로 만들어줍니다.
    cur_user_data = (new_df_counts[new_df_counts['user']==user['user_id']]
                     .set_index('category')
                     .rename({'count': 'sum'}, axis=1)
                     .drop(columns=['user']))
    
    # 그룹화 추천 결과와 유저 정보를 더합니다.
    # 개인이 조회한 횟수 + 유저 그룹이 평균적으로 조회한 횟수
    grouped_view_df = (grouped_view_df.add(cur_user_data, fill_value=0)
                       .sort_values(by='sum', ascending=False)
                       .reset_index())
    return grouped_view_df

# user_id는 2인 22세 남자 데이터입니다.
# 아까 맨 처음에 넣은 DB 데이터의 정보를 가진 유저입니당
tester2 = {'user_id': 2, 'gender': '남자', 'age': 22}

# 같은 남성, 20대인데.... 결과는 이 유저는 남성 하의, 상의를 자주보니까 달라졌네요!
result = recom_simple2(tester2)
result['category'] = result['category'].apply(lambda x: category_dict[x])
print(result)

       category     sum
0         남성 하의  2.9875
1         남성 상의  2.0625
2        스마트 밴드  1.4500
3        스마트 워치  1.2500
4       남성 캐주얼화  1.1500
..          ...     ...
145  기타 스케이트 용품  0.2125
146       남성 향수  0.2125
147         전동휠  0.2000
148          목공  0.2000
149  기타 헤어 스타일링  0.1375

[150 rows x 2 columns]


In [7]:
# 각각의 그룹에 대한 기본 추천 데이터입니다.
testers = [{'gender': '남자', 'age': 10}, {'gender': '남자', 'age': 20}, {'gender': '남자', 'age': 30}, {'gender': '남자', 'age': 40}, {'gender': '남자', 'age': 50}, {'gender': '남자', 'age': 60}, {'gender': '남자', 'age': 70},
           {'gender': '여자', 'age': 10}, {'gender': '여자', 'age': 20}, {'gender': '여자', 'age': 30}, {'gender': '여자', 'age': 40}, {'gender': '여자', 'age': 50}, {'gender': '여자', 'age': 60}, {'gender': '여자', 'age': 70}]

for test in testers:
    print(test['gender'], test['age'])
    result1 = recom_simple1(test)
    result1['category'] = result1['category'].apply(lambda x: category_dict[x])
    print(result1)

남자 10
    category     sum
0     남성 운동화  1.2250
1     남성용 가방  1.2125
2      남성 하의  1.1875
3        야구공  1.1375
4    남성 캐주얼화  1.1250
..       ...     ...
89        잡지  0.0875
90     축구 의류  0.0875
91       탁구대  0.0875
92  기타 축구 용품  0.0875
93     축구 가방  0.0750

[94 rows x 2 columns]
남자 20
       category     sum
0        스마트 밴드  1.4500
1        스마트 워치  1.2500
2       남성 캐주얼화  1.1500
3        남성 운동화  1.1500
4        남성용 모자  1.1000
..          ...     ...
144  기타 스케이트 용품  0.2125
145       남성 향수  0.2125
146         전동휠  0.2000
147          목공  0.2000
148  기타 헤어 스타일링  0.1375

[149 rows x 2 columns]
남자 30
      category       sum
0       스마트 밴드  1.650000
1       스마트 워치  1.466667
2          키보드  1.216667
3     기타 음향 가전  1.200000
4          청소기  1.183333
..         ...       ...
143         헹거  0.266667
144     아동 서랍장  0.250000
145  기타 DIY 자재  0.250000
146     주방 수납장  0.216667
147      책상/의자  0.216667

[148 rows x 2 columns]
남자 40
      category       sum
0     문학/과학/경영  1.566667
1        문구 용품 

In [8]:
#####################################
# 이하는 Model 정확도 측정입니다.   #
#####################################

In [9]:
from sklearn.model_selection import train_test_split

merged_df = pd.merge(df_counts, df_users, left_on='user', right_on='user')
x = merged_df.copy()
y = merged_df['user']
# Model을 테스트하기 위해 75%는 train set / 25%는 test set으로 나눕니다.
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, stratify=y)

In [10]:
# RMSE 계산해주는 함수
def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true)- np.array(y_pred)) ** 2))

# 모델별 RMSE 계산
def score(model):
    # test set에 있는 user랑 category를 합쳐줍니다.
    id_pairs = zip(x_test['user'], x_test['category'])
    # 각각의 user, category로 예측값을 계산합니다.
    y_pred = np.array([model(user, category) for (user, category) in id_pairs])
    # 실제값을 가져옵니다.
    y_true = np.array(x_test['count'])
    return RMSE(y_true, y_pred)

train_view_matrix = x_train.pivot(index='user', columns='category', values='count')
view_matrix_t = np.transpose(train_view_matrix)

In [11]:
def simple_all_model(user, category):
    # 이거 코드를 좀 이상하게 짰는데요... 일단 이렇게 써도 돌아는 가요 (좀 오래 걸려요)
    # 현재 유저 데이터를 찾습니다. (user_no로 user 정보 획득)
    cur_user = new_df_users[new_df_users['user'] == user]
    try:
        # 만약 특정 카테고리에 대해 사용자가 속한 그룹의 사람들이 조회한 기록이 있다면,
        # 그 평균값을 예측값으로 합니다.
        pred_viewing = train_mean.loc[cur_user['gender'].values[0]].loc[cur_user['age'].values[0]].loc[category]['mean']
    except:
        # 만약 특정 카테고리에 대해 사용자가 속한 그룹의 사람들이 한 명도 안 봤다면
        # 해당 카테고리의 전체 평균 조뢰수를 가져옵니다.
        pred_viewing = train_mean_all[category]
    return pred_viewing

user_count = len(x_train['user'].unique())
train_mean_all = x_train.groupby(['category'])['count'].sum().apply(lambda x: x/user_count)

train_mean = (x_train.groupby(['gender', 'age', 'category'])['count']
              .agg(['sum', 'size']))
train_mean['mean'] = train_mean['sum'] / train_mean['size']

print(score(simple_all_model))

1.1467279863327897
